# NN, LSTM
example from: http://philipperemy.github.io/keras-stateful-lstm/

In [330]:
%reset -f
import numpy as np
from keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [348]:
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)

## Data

## test data
random 0/1 in X[num_samples,time_steps] <br>
expected result Y=X[:,0] <br>
for example X=[[1,0],[0,0],[0,0],[1,0]], Y=[1,0,0,1]. <br>
data is fed into the LSTM classifier in smaller pieces than "time_steps", hence the classifier has to remember it's previous state to be better than just guessing (p=0.5).

In [331]:
time_steps = 20
num_train_samples = 1000
num_test_samples = 200

In [332]:
# Pre-allocating the numpy array for better readability
X_train = np.zeros((num_train_samples, time_steps, 1), dtype=int)
X_test = np.zeros((num_test_samples, time_steps, 1), dtype=int)
y_train = np.zeros((num_train_samples), dtype=int)
y_test = np.zeros((num_test_samples), dtype=int)

# Setting internal time-steps to random numbers
X_train[:, 1:] = np.random.randint(0, 2, (num_train_samples, time_steps-1, 1), dtype=int)
X_test[:, 1:] = np.random.randint(0, 2, (num_test_samples, time_steps-1, 1), dtype=int)
# Setting half of the first time-steps to 1
one_indexes = np.random.choice(a=num_train_samples, size=int(num_train_samples / 2), replace=False)
X_train[one_indexes, 0] = 1
one_indexes = np.random.choice(a=num_test_samples, size=int(num_test_samples / 2), replace=False)
X_test[one_indexes, 0] = 1

# Creating labels
y_train = X_train[:, 0, 0]
y_test = X_test[:, 0, 0]

In [333]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1000, 20, 1), (1000,), (200, 20, 1), (200,))

In [334]:
X_train[0:5,:,0], y_train[0:5]

(array([[0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1],
        [1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1],
        [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0],
        [1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0],
        [0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0]]),
 array([0, 1, 1, 1, 0]))

In [349]:
X_train_sub, y_train_sub = prepare_sequences(X_train, y_train, window_length=10)
X_test_sub, y_test_sub = prepare_sequences(X_test, y_test, window_length=10)

In [350]:
X_train_sub.shape, y_train_sub.shape, X_test_sub.shape, y_test_sub.shape

((11000, 10, 1), (11000,), (2200, 10, 1), (2200,))

In [351]:
X_train_sub[(0,11,22,33,44,55),0], y_train_sub[(0,11,22,33,44,55),]

(array([[0],
        [1],
        [1],
        [1],
        [0],
        [1]]),
 array([0, 1, 1, 1, 0, 1]))

In [363]:
x = np.expand_dims(np.expand_dims(X_train.flatten(), axis=1), axis=1)
y = np.expand_dims(np.array([[v] * 20 for v in y_train.flatten()]).flatten(), axis=1)
xx = np.expand_dims(np.expand_dims(X_test.flatten(), axis=1), axis=1)
yy = np.expand_dims(np.array([[v] * 20 for v in y_test.flatten()]).flatten(), axis=1)

In [364]:
x.shape, y.shape, xx.shape, yy.shape

((20000, 1, 1), (20000, 1), (4000, 1, 1), (4000, 1))

In [366]:
x[0:10,0,0], y[0:10,0]

(array([0, 0, 0, 1, 0, 0, 1, 0, 1, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

## simple LSTM model
does not remember states between samples, not effective

In [369]:
m = Sequential()
m.add(LSTM(5, input_shape=(20, 1), stateful=False))
m.add(Dense(1, activation='sigmoid'))
#m.compile(loss='MSE', optimizer='adam', metrics=['accuracy'])
m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(m.summary())
m.fit(X_train, y_train, batch_size=1, epochs=5, shuffle=False, validation_data=(X_test, y_test))

Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_46 (LSTM)               (None, 5)                 140       
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 6         
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
None
Train on 1000 samples, validate on 200 samples
Epoch 1/5
1000/1000 [==============================] - 7s 7ms/sample - loss: 0.6949 - accuracy: 0.4800 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 2/5
1000/1000 [==============================] - 5s 5ms/sample - loss: 0.6945 - accuracy: 0.4770 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 3/5
1000/1000 [==============================] - 5s 5ms/sample - loss: 0.6940 - accuracy: 0.4890 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 4/5
1000/1000 [=========

In [375]:
m = Sequential()
m.add(LSTM(5, input_shape=(10, 1), stateful=False))
m.add(Dense(1, activation='sigmoid'))
#m.compile(loss='MSE', optimizer='adam', metrics=['accuracy'])
m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(m.summary())
m.fit(X_train_sub, y_train_sub, batch_size=11, epochs=5, shuffle=False, validation_data=(X_test_sub, y_test_sub))

Model: "sequential_69"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, 5)                 140       
_________________________________________________________________
dense_77 (Dense)             (None, 1)                 6         
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
None
Train on 11000 samples, validate on 2200 samples
Epoch 1/5
11000/11000 [==============================] - 6s 558us/sample - loss: 0.6944 - accuracy: 0.5012 - val_loss: 0.6947 - val_accuracy: 0.5000
Epoch 2/5
11000/11000 [==============================] - 5s 434us/sample - loss: 0.6946 - accuracy: 0.4856 - val_loss: 0.6945 - val_accuracy: 0.5000
Epoch 3/5
11000/11000 [==============================] - 5s 410us/sample - loss: 0.6945 - accuracy: 0.4877 - val_loss: 0.6943 - val_accuracy: 0.5000
Epoch 4/5
11000/

In [376]:
m = Sequential()
m.add(LSTM(5, input_shape=(1, 1), stateful=False))
m.add(Dense(1, activation='relu'))
m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(m.summary())
m.fit(x, y, batch_size=20, epochs=5, shuffle=False, validation_data=(xx, yy))

Model: "sequential_70"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_53 (LSTM)               (None, 5)                 140       
_________________________________________________________________
dense_78 (Dense)             (None, 1)                 6         
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
None
Train on 20000 samples, validate on 4000 samples
Epoch 1/5
20000/20000 [==============================] - 6s 298us/sample - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 2/5
20000/20000 [==============================] - 4s 212us/sample - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 3/5
20000/20000 [==============================] - 4s 219us/sample - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 4/5
20000/

## Simple NN
simplest model, works for this simple example by connectin x[0] to y

In [458]:
m = Sequential()
m.add(Flatten(input_shape=(time_steps, 1)))
m.add(Dense(5, activation='relu'))
m.add(Dense(1, activation='sigmoid'))
m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(m.summary())
m.fit(X_train, y_train, batch_size=20, epochs=15, shuffle=False, validation_data=(X_test, y_test))

Model: "sequential_94"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 20)                0         
_________________________________________________________________
dense_102 (Dense)            (None, 5)                 105       
_________________________________________________________________
dense_103 (Dense)            (None, 1)                 6         
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
None
Train on 1000 samples, validate on 200 samples
Epoch 1/15
1000/1000 [==============================] - 1s 569us/sample - loss: 0.7093 - accuracy: 0.5360 - val_loss: 0.6746 - val_accuracy: 0.5350
Epoch 2/15
1000/1000 [==============================] - 0s 180us/sample - loss: 0.6559 - accuracy: 0.6050 - val_loss: 0.6420 - val_accuracy: 0.6300
Epoch 3/15
1000/1000 [===============

## stateful LSTM
effective with storing state inside 20 samples and reset of values afterwards<br>

In [391]:
print('Build STATEFUL model...')
model = Sequential()
model.add(LSTM(10, batch_input_shape=(1, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Build STATEFUL model...


In [392]:

class MyCustomCallback(Callback):
    def __init__(self):
        #self.counter = 0
        return None
    
    def on_train_batch_begin(self, batch, logs=None):
        if batch%20 == 0:
            self.model.reset_states()

    def on_train_batch_end(self, batch, logs=None):
        return None

    def on_test_batch_begin(self, batch, logs=None):
        if batch%20 == 0:
            self.model.reset_states()

    def on_test_batch_end(self, batch, logs=None):
        return None
    
    def on_test_begin(self, batch, logs=None):
        return None
    
    def on_test_end(self, batch, logs=None):
        return None


In [393]:
model.fit(x, y, batch_size=1, epochs=2, validation_data=(xx, yy), shuffle=False, callbacks=[MyCustomCallback()])

Train on 20000 samples, validate on 4000 samples
Epoch 1/2
20000/20000 [==============================] - 92s 5ms/sample - loss: 0.4736 - accuracy: 0.7002 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 2/2
20000/20000 [==============================] - 84s 4ms/sample - loss: 1.3370e-04 - accuracy: 1.0000 - val_loss: 6.0619e-08 - val_accuracy: 1.0000
